In [14]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (12,8)

In [15]:
df = pd.read_csv('data/raw/bangkok_traffy.csv')
print(df.shape)
df.head()

(787026, 16)


,ticket_id,type,organization,comment,photo,photo_after,coords,address,subdistrict,district,province,timestamp,state,star,count_reopen,last_activity
0,2021-FYJTFP,{ความสะอาด},เขตบางซื่อ,ขยะเยอะ,https://storage.googleapis.com/traffy_public_b...,NaN,"100.53084,13.81865",12/14 ถนน กรุงเทพ- นนทบุรี แขวง บางซื่อ เขตบาง...,NaN,NaN,กรุงเทพมหานคร,2021-09-03 12:51:09.453003+00,เสร็จสิ้น,NaN,0,2022-06-04 15:34:14.609206+00
1,2021-CGPMUN,"{น้ำท่วม,ร้องเรียน}","เขตประเวศ,ฝ่ายโยธา เขตประเวศ",น้ำท่วมเวลาฝนตกและทะลุเข้าบ้านเดือดร้อนมากทุกๆ...,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,"100.66709,13.67891",189 เฉลิมพระเกียรติ ร.9 แขวง หนองบอน เขต ประเว...,หนองบอน,ประเวศ,กรุงเทพมหานคร,2021-09-19 14:56:08.924992+00,เสร็จสิ้น,4.0,0,2022-06-21 08:21:09.532782+00
2,2021-7XATFA,{สะพาน},เขตสาทร,สะพานลอยปรับปรุงไม่เสร็จตามกำหนด\nปากซอย สาทร12,https://storage.googleapis.com/traffy_public_b...,NaN,"100.52649,13.72060",191/1 ถนน สาทรเหนือ แขวง สีลม เขตบางรัก กรุงเท...,ยานนาวา,สาทร,กรุงเทพมหานคร,2021-09-26 05:03:52.594898+00,เสร็จสิ้น,NaN,0,2022-06-06 01:17:12.272904+00
3,2021-9U2NJT,{น้ำท่วม},"เขตบางซื่อ,ฝ่ายโยธา เขตบางซื่อ",น้ำท่วม,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,"100.53099,13.81853",12/14 ถนน กรุงเทพ- นนทบุรี แขวง บางซื่อ เขตบาง...,NaN,NaN,กรุงเทพมหานคร,2021-10-14 10:45:27.713884+00,เสร็จสิ้น,NaN,0,2022-09-08 08:35:43.784519+00
4,2021-DVEWYM,"{น้ำท่วม,ถนน}","เขตลาดพร้าว,ฝ่ายโยธา เขตลาดพร้าว",ซอยลาดพร้าววังหิน 75 ถนนลาดพร้าววังหิน แขวงลาด...,https://storage.googleapis.com/traffy_public_b...,NaN,"100.59165,13.82280",702 ถ. ลาดพร้าววังหิน แขวงลาดพร้าว เขตลาดพร้าว...,ลาดพร้าว,ลาดพร้าว,กรุงเทพมหานคร,2021-12-09 12:29:08.408763+00,เสร็จสิ้น,5.0,0,2022-08-12 07:18:44.884945+00


In [16]:
weather_df = pd.read_csv('data/raw/open-meteo-13.74N100.50E9m.csv')
# D:\CEDT\02_01\data_Sci\dsde-final\data\raw\open-meteo-13.74N100.50E9m.csv
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30696 entries, 0 to 30695
Data columns (total 10 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   time                           30696 non-null  object 
 1   temperature_2m (°C)            30696 non-null  float64
 2   dew_point_2m (°C)              30696 non-null  float64
 3   relative_humidity_2m (%)       30696 non-null  int64  
 4   rain (mm)                      30696 non-null  float64
 5   vapour_pressure_deficit (kPa)  30696 non-null  float64
 6   cloud_cover (%)                30696 non-null  int64  
 7   wind_direction_10m (°)         30696 non-null  int64  
 8   surface_pressure (hPa)         30696 non-null  float64
 9   wind_speed_10m (km/h)          30696 non-null  float64
dtypes: float64(6), int64(3), object(1)
memory usage: 2.3+ MB


In [17]:
traffy = df.drop(['ticket_id', 'organization', 'photo', 'photo_after',
            'address', 'state', 'star', 'count_reopen', 'last_activity'], axis=1)

traffy.dropna(inplace=True)

traffy = traffy[traffy['province'] == 'กรุงเทพมหานคร'].reset_index(drop=True)

In [18]:
traffy.info()
traffy['province'].mode()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 651600 entries, 0 to 651599
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   type         651600 non-null  object
 1   comment      651600 non-null  object
 2   coords       651600 non-null  object
 3   subdistrict  651600 non-null  object
 4   district     651600 non-null  object
 5   province     651600 non-null  object
 6   timestamp    651600 non-null  object
dtypes: object(7)
memory usage: 34.8+ MB


0    กรุงเทพมหานคร
Name: province, dtype: object

In [19]:
traffy.head()

,type,comment,coords,subdistrict,district,province,timestamp
0,"{น้ำท่วม,ร้องเรียน}",น้ำท่วมเวลาฝนตกและทะลุเข้าบ้านเดือดร้อนมากทุกๆ...,"100.66709,13.67891",หนองบอน,ประเวศ,กรุงเทพมหานคร,2021-09-19 14:56:08.924992+00
1,{สะพาน},สะพานลอยปรับปรุงไม่เสร็จตามกำหนด\nปากซอย สาทร12,"100.52649,13.72060",ยานนาวา,สาทร,กรุงเทพมหานคร,2021-09-26 05:03:52.594898+00
2,"{น้ำท่วม,ถนน}",ซอยลาดพร้าววังหิน 75 ถนนลาดพร้าววังหิน แขวงลาด...,"100.59165,13.82280",ลาดพร้าว,ลาดพร้าว,กรุงเทพมหานคร,2021-12-09 12:29:08.408763+00
3,{},หน้าปากซอย ลาดพร้าววังหิน26,"100.59131,13.80910",ลาดพร้าว,ลาดพร้าว,กรุงเทพมหานคร,2021-12-13 05:53:36.861064+00
4,{},ยังไม่มีหน่วยงานไหนมาดูแลครับ รถจะเชี่ยวหลายคน...,"100.50848,13.77832",ดุสิต,ดุสิต,กรุงเทพมหานคร,2021-12-17 08:46:02.610983+00


In [21]:
merge_df = pd.read_csv('data/processed/traffy_weather_merged.csv')
merge_df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'data/processed/traffy_weather_merged.csv'

In [22]:
# Save the cleaned `traffy` DataFrame to disk
import os
os.makedirs('data/processed', exist_ok=True)
traffy.to_csv('data/processed/traffy_clean.csv', index=False)
print('Saved: data/processed/traffy_clean.csv')

Saved: data/processed/traffy_clean.csv


In [23]:
clean_df = pd.read_csv('data/processed/traffy_clean.csv')

In [24]:
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 651600 entries, 0 to 651599
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   type         651600 non-null  object
 1   comment      651600 non-null  object
 2   coords       651600 non-null  object
 3   subdistrict  651600 non-null  object
 4   district     651600 non-null  object
 5   province     651600 non-null  object
 6   timestamp    651600 non-null  object
dtypes: object(7)
memory usage: 34.8+ MB


In [ ]:
clean_df.head()

,type,comment,coords,subdistrict,district,province,timestamp
0,"{น้ำท่วม,ร้องเรียน}",น้ำท่วมเวลาฝนตกและทะลุเข้าบ้านเดือดร้อนมากทุกๆ...,"100.66709,13.67891",หนองบอน,ประเวศ,กรุงเทพมหานคร,2021-09-19 14:56:08.924992+00
1,{สะพาน},สะพานลอยปรับปรุงไม่เสร็จตามกำหนด\nปากซอย สาทร12,"100.52649,13.72060",ยานนาวา,สาทร,กรุงเทพมหานคร,2021-09-26 05:03:52.594898+00
2,"{น้ำท่วม,ถนน}",ซอยลาดพร้าววังหิน 75 ถนนลาดพร้าววังหิน แขวงลาด...,"100.59165,13.82280",ลาดพร้าว,ลาดพร้าว,กรุงเทพมหานคร,2021-12-09 12:29:08.408763+00
3,{},หน้าปากซอย ลาดพร้าววังหิน26,"100.59131,13.80910",ลาดพร้าว,ลาดพร้าว,กรุงเทพมหานคร,2021-12-13 05:53:36.861064+00
4,{},ยังไม่มีหน่วยงานไหนมาดูแลครับ รถจะเชี่ยวหลายคน...,"100.50848,13.77832",ดุสิต,ดุสิต,กรุงเทพมหานคร,2021-12-17 08:46:02.610983+00


In [26]:
air_df = pd.read_csv('data/processed/bangkok-air-quality.csv')
air_df.head()

,date,pm25,pm10,o3,no2,so2,co
0,2025/11/1,58,16,9,10,,
1,2025/11/2,56,13,10,4,,
2,2025/11/3,49,16,15,8,,
3,2025/11/4,58,20,16,11,,
4,2025/11/5,61,25,18,9,,


In [27]:
print(air_df.columns.tolist())

['date', ' pm25', ' pm10', ' o3', ' no2', ' so2', ' co']


In [28]:
air_df = air_df.drop([' so2',' co'], axis=1)

In [29]:
air_df.head()

,date,pm25,pm10,o3,no2
0,2025/11/1,58,16,9,10
1,2025/11/2,56,13,10,4
2,2025/11/3,49,16,15,8
3,2025/11/4,58,20,16,11
4,2025/11/5,61,25,18,9


In [30]:
# clean_df['date'] = pd.to_datetime(clean_df['timestamp']).dt.strftime('%Y-%m-%d')
clean_df['date'] = pd.to_datetime(clean_df['timestamp'], format='ISO8601').dt.date


In [31]:
clean_df['date'] = pd.to_datetime(clean_df['date']).dt.normalize()

In [32]:
clean_df.info()
clean_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 651600 entries, 0 to 651599
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   type         651600 non-null  object        
 1   comment      651600 non-null  object        
 2   coords       651600 non-null  object        
 3   subdistrict  651600 non-null  object        
 4   district     651600 non-null  object        
 5   province     651600 non-null  object        
 6   timestamp    651600 non-null  object        
 7   date         651600 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(7)
memory usage: 39.8+ MB


,type,comment,coords,subdistrict,district,province,timestamp,date
0,"{น้ำท่วม,ร้องเรียน}",น้ำท่วมเวลาฝนตกและทะลุเข้าบ้านเดือดร้อนมากทุกๆ...,"100.66709,13.67891",หนองบอน,ประเวศ,กรุงเทพมหานคร,2021-09-19 14:56:08.924992+00,2021-09-19
1,{สะพาน},สะพานลอยปรับปรุงไม่เสร็จตามกำหนด\nปากซอย สาทร12,"100.52649,13.72060",ยานนาวา,สาทร,กรุงเทพมหานคร,2021-09-26 05:03:52.594898+00,2021-09-26
2,"{น้ำท่วม,ถนน}",ซอยลาดพร้าววังหิน 75 ถนนลาดพร้าววังหิน แขวงลาด...,"100.59165,13.82280",ลาดพร้าว,ลาดพร้าว,กรุงเทพมหานคร,2021-12-09 12:29:08.408763+00,2021-12-09
3,{},หน้าปากซอย ลาดพร้าววังหิน26,"100.59131,13.80910",ลาดพร้าว,ลาดพร้าว,กรุงเทพมหานคร,2021-12-13 05:53:36.861064+00,2021-12-13
4,{},ยังไม่มีหน่วยงานไหนมาดูแลครับ รถจะเชี่ยวหลายคน...,"100.50848,13.77832",ดุสิต,ดุสิต,กรุงเทพมหานคร,2021-12-17 08:46:02.610983+00,2021-12-17


In [33]:
air_df['date'] = pd.to_datetime(air_df['date']).dt.normalize()

In [34]:
air_df.info()
air_df.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4341 entries, 0 to 4340
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    4341 non-null   datetime64[ns]
 1    pm25   4341 non-null   object        
 2    pm10   4341 non-null   object        
 3    o3     4341 non-null   object        
 4    no2    4341 non-null   object        
dtypes: datetime64[ns](1), object(4)
memory usage: 169.7+ KB


,date,pm25,pm10,o3,no2
0,2025-11-01,58,16,9,10
1,2025-11-02,56,13,10,4
2,2025-11-03,49,16,15,8
3,2025-11-04,58,20,16,11
4,2025-11-05,61,25,18,9


In [35]:
merged_df = clean_df.merge(air_df, on='date', how='left')

In [36]:
merged_df.info()
merged_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 651600 entries, 0 to 651599
Data columns (total 12 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   type         651600 non-null  object        
 1   comment      651600 non-null  object        
 2   coords       651600 non-null  object        
 3   subdistrict  651600 non-null  object        
 4   district     651600 non-null  object        
 5   province     651600 non-null  object        
 6   timestamp    651600 non-null  object        
 7   date         651600 non-null  datetime64[ns]
 8    pm25        651138 non-null  object        
 9    pm10        651138 non-null  object        
 10   o3          651138 non-null  object        
 11   no2         651138 non-null  object        
dtypes: datetime64[ns](1), object(11)
memory usage: 59.7+ MB


,type,comment,coords,subdistrict,district,province,timestamp,date,pm25,pm10,o3,no2
0,"{น้ำท่วม,ร้องเรียน}",น้ำท่วมเวลาฝนตกและทะลุเข้าบ้านเดือดร้อนมากทุกๆ...,"100.66709,13.67891",หนองบอน,ประเวศ,กรุงเทพมหานคร,2021-09-19 14:56:08.924992+00,2021-09-19,52,27,9,4
1,{สะพาน},สะพานลอยปรับปรุงไม่เสร็จตามกำหนด\nปากซอย สาทร12,"100.52649,13.72060",ยานนาวา,สาทร,กรุงเทพมหานคร,2021-09-26 05:03:52.594898+00,2021-09-26,25,18,8,6
2,"{น้ำท่วม,ถนน}",ซอยลาดพร้าววังหิน 75 ถนนลาดพร้าววังหิน แขวงลาด...,"100.59165,13.82280",ลาดพร้าว,ลาดพร้าว,กรุงเทพมหานคร,2021-12-09 12:29:08.408763+00,2021-12-09,100,54,20,15
3,{},หน้าปากซอย ลาดพร้าววังหิน26,"100.59131,13.80910",ลาดพร้าว,ลาดพร้าว,กรุงเทพมหานคร,2021-12-13 05:53:36.861064+00,2021-12-13,79,65,23,14
4,{},ยังไม่มีหน่วยงานไหนมาดูแลครับ รถจะเชี่ยวหลายคน...,"100.50848,13.77832",ดุสิต,ดุสิต,กรุงเทพมหานคร,2021-12-17 08:46:02.610983+00,2021-12-17,117,48,18,6


In [38]:
merged_df['type'] = merged_df['type'].replace("{}", "ไม่ระบุ")
import ast

def to_list(x):
    if x == "ไม่ระบุ" or x == "{}":
        return []
    # แปลง "{น้ำท่วม,ถนน}" → ["น้ำท่วม","ถนน"]
    return [i.strip() for i in x.strip("{}").split(",")]

merged_df['type_list'] = merged_df['type'].apply(to_list)

os.makedirs('data/processed', exist_ok=True)
merged_df.to_csv('data/processed/traffy_merged.csv', index=False)
print('Saved: data/processed/traffy_merged.csv')

Saved: data/processed/traffy_merged.csv


In [40]:
df_t= pd.read_csv('data/processed/traffy_merged.csv')
# df_t.shape
df_t.head()

C:\Users\jkuny\AppData\Local\Temp\ipykernel_22476\2761252754.py:1: DtypeWarning: Columns (8,9,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df_t= pd.read_csv('data/processed/traffy_merged.csv')


,type,comment,coords,subdistrict,district,province,timestamp,date,pm25,pm10,o3,no2,type_list
0,"{น้ำท่วม,ร้องเรียน}",น้ำท่วมเวลาฝนตกและทะลุเข้าบ้านเดือดร้อนมากทุกๆ...,"100.66709,13.67891",หนองบอน,ประเวศ,กรุงเทพมหานคร,2021-09-19 14:56:08.924992+00,2021-09-19,52,27,9,4,"['น้ำท่วม', 'ร้องเรียน']"
1,{สะพาน},สะพานลอยปรับปรุงไม่เสร็จตามกำหนด\nปากซอย สาทร12,"100.52649,13.72060",ยานนาวา,สาทร,กรุงเทพมหานคร,2021-09-26 05:03:52.594898+00,2021-09-26,25,18,8,6,['สะพาน']
2,"{น้ำท่วม,ถนน}",ซอยลาดพร้าววังหิน 75 ถนนลาดพร้าววังหิน แขวงลาด...,"100.59165,13.82280",ลาดพร้าว,ลาดพร้าว,กรุงเทพมหานคร,2021-12-09 12:29:08.408763+00,2021-12-09,100,54,20,15,"['น้ำท่วม', 'ถนน']"
3,ไม่ระบุ,หน้าปากซอย ลาดพร้าววังหิน26,"100.59131,13.80910",ลาดพร้าว,ลาดพร้าว,กรุงเทพมหานคร,2021-12-13 05:53:36.861064+00,2021-12-13,79,65,23,14,[]
4,ไม่ระบุ,ยังไม่มีหน่วยงานไหนมาดูแลครับ รถจะเชี่ยวหลายคน...,"100.50848,13.77832",ดุสิต,ดุสิต,กรุงเทพมหานคร,2021-12-17 08:46:02.610983+00,2021-12-17,117,48,18,6,[]
